In [0]:
create or replace table experiment_sequential_results as 

WITH
-- 1) Find transaction number per arm
  ranked_txn as (
    SELECT
      category,
      experiment_group,
      amount,
      row_number() over(partition by category, experiment_group order by date) as txn_number
    from
      money_mop.silver_daily_transactions
  ),
  -- 2) Match transaction number with plan
  matched_checkpoints as (
    SELECT
      t.category,
      t.experiment_group,
      t.txn_number,
      t.amount,
      (
        SELECT p.checkpoint_num
        FROM money_mop.experiment_sequential_plan p
        WHERE p.category = t.category
          AND t.txn_number <= p.transactions_req
        ORDER BY p.transactions_req
        LIMIT 1
      ) AS checkpoint_num,
      (
        SELECT p.transactions_req
        FROM money_mop.experiment_sequential_plan p
        WHERE p.category = t.category
          AND t.txn_number <= p.transactions_req
        ORDER BY p.transactions_req
        LIMIT 1
      ) AS transactions_req
    FROM ranked_txn t
    ORDER BY t.experiment_group, t.txn_number
  ),
  -- 3) Join with plan
  txn_with_plan as (
    SELECT
      m.category,
      m.experiment_group,
      m.txn_number,
      m.amount,
      m.checkpoint_num,
      m.transactions_req,
      p.information_frac,
      p.efficacy_z,
      p.futility_z,
      p.harm_z
    FROM matched_checkpoints m
    JOIN money_mop.experiment_sequential_plan p
      ON m.category         = p.category
    AND m.checkpoint_num    = p.checkpoint_num
  ),
  -- 4) Aggregation
  arm_stats as (
    SELECT
      category,
      experiment_group,
      checkpoint_num,
      transactions_req,
      information_frac,
      efficacy_z,
      futility_z,
      harm_z,
      count(*) as n,
      avg(amount) as mean_spend,
      stddev_samp(amount) as std_spend
    from
      txn_with_plan
    group by
      category,
      experiment_group,
      checkpoint_num,
      transactions_req,
      information_frac,
      efficacy_z,
      futility_z,
      harm_z
  ),
  -- 5) pivots
  pivoted as (
    select    
      category,
      checkpoint_num,
      transactions_req,
      information_frac,
      efficacy_z,
      futility_z,
      harm_z,
      MAX(CASE WHEN experiment_group='A' THEN n        END) AS n_A,
      MAX(CASE WHEN experiment_group='A' THEN mean_spend END) AS mean_A,
      MAX(CASE WHEN experiment_group='A' THEN std_spend  END) AS std_A,
      MAX(CASE WHEN experiment_group='B' THEN n        END) AS n_B,
      MAX(CASE WHEN experiment_group='B' THEN mean_spend END) AS mean_B,
      MAX(CASE WHEN experiment_group='B' THEN std_spend  END) AS std_B
    FROM arm_stats
    GROUP BY
      category,
      checkpoint_num,
      transactions_req,
      information_frac,
      efficacy_z,
      futility_z,
      harm_z
  )
  SELECT
    category,
    checkpoint_num,
    transactions_req,
    information_frac,
    n_A, n_B,
    mean_A, mean_B,
    std_A, std_B,

    -- observed Z
    (mean_B - mean_A)
      / SQRT( POWER(std_A,2)/n_A + POWER(std_B,2)/n_B)
      AS z_statistic,

    efficacy_z,
    futility_z,
    harm_z,

    -- decision based on boundaries
    CASE
      WHEN (mean_B - mean_A)
          / SQRT( POWER(std_A,2)/n_A + POWER(std_B,2)/n_B) >= efficacy_z
        THEN 'efficacy'
      WHEN (mean_B - mean_A)
          / SQRT( POWER(std_A,2)/n_A + POWER(std_B,2)/n_B) <= futility_z
        THEN 'futility'
      ELSE 'continue'
    END AS decision

  FROM pivoted
  ORDER BY category, checkpoint_num

